In [1]:
from aidds.sys.utils.exception import AiddsException
from aidds.modeling.scaler import Scaling 
from aidds.modeling.learn import Learning
import aidds.sys.config as cfg
from aidds.sys.utils.evaluation import regression_evals
import pandas as pd

In [2]:
try:
    sc = Scaling(is_best=True)
    ln = Learning(scaling_data=sc.sdata)
except AiddsException as ae:
    ae.print()

[0a98f214f2c0][2024-04-17 14:46:56.300995] Preprocessing data scaling start.
[0a98f214f2c0][2024-04-17 14:46:56.423537]   Total size of attribute data for learning: (10963, 98)
[0a98f214f2c0][2024-04-17 14:46:56.428912]   Data size of training/testing data by pole count: pc_type[all] - total(10963, 98), train(8222, 98), test(2741, 98)
[0a98f214f2c0][2024-04-17 14:46:56.454401]   Data size of training/testing data by pole count: pc_type[e1] - total(6635, 98), train(4976, 98), test(1659, 98)
[0a98f214f2c0][2024-04-17 14:46:56.477190]   Data size of training/testing data by pole count: pc_type[n1] - total(4328, 98), train(3246, 98), test(1082, 98)
[0a98f214f2c0][2024-04-17 14:47:00.349435] Preprocessing data scaling end., Total processing time: 0:00:04.048421
[45b8ed18cd34][2024-04-17 14:47:00.349997] Learning start.


/home/freeman/anaconda3/envs/aidds/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.382e+15, tolerance: 8.284e+12
  model = cd_fast.enet_coordinate_descent(
/home/freeman/anaconda3/envs/aidds/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.628e+14, tolerance: 5.777e+11
  model = cd_fast.enet_coordinate_descent(
/home/freeman/anaconda3/envs/aidds/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

{'all': {'model': GradientBoostingRegressor(), 'score': 0.9116274609946394, 'mape': 0.10390456815923603, 'model_key': 'gbr'}, 'e1': {'model': RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=1234), 'score': 0.7491454709387149, 'mape': 0.10408537684781527, 'model_key': 'rfr'}, 'n1': {'model': RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=1234), 'score': 0.8526525655217254, 'mape': 0.1152456588322366, 'model_key': 'rfr'}}
-0.3868063734903075
[45b8ed18cd34][2024-04-17 14:47:13.017219]   Model training results: [모델 그대로 사용] - MAPE: 0.103905, R2SCORE: 0.911627
[45b8ed18cd34][2024-04-17 14:47:13.023595]   Model training results: [읽어들인 모델 사용] - MAPE: 0.103905, R2SCORE: 0.911627
-0.3868063734903075
{'all': {'model': GradientBoostingRegressor(), 'score': 0.9116274609946394, 'mape': 0.10390456815923603, 'model_key': 'gbr'}, 'e1': {'model': RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=1234), 'score': 0.7491454709387149, 'mape': 0.1040853768478152

In [3]:
ln._best

{'all': {'model': GradientBoostingRegressor(),
  'score': 0.9116274609946394,
  'mape': 0.10390456815923603,
  'model_key': 'gbr'},
 'e1': {'model': GradientBoostingRegressor(),
  'score': 0.7734945176988901,
  'mape': 0.09855364022973306,
  'model_key': 'gbr'},
 'n1': {'model': GradientBoostingRegressor(),
  'score': 0.8712469407446384,
  'mape': 0.11173733296563843,
  'model_key': 'gbr'}}

In [4]:
model = ln._best['all']['model']
ddict = {did: ln._sdict[did]['all'] for did in cfg.type.mds.ids}

In [5]:
ddict.keys()

dict_keys(['train_x', 'test_x', 'train_y', 'test_y'])

In [6]:
type(ddict['train_x'])

pandas.core.frame.DataFrame

In [7]:
# train_x = pd.concat([ddict['train_x'], ddict['test_x']])
# train_y = pd.concat([ddict['train_y'], ddict['test_y']])
train_x = ddict['test_x']
train_y = ddict['test_y']

In [8]:
train_x.shape

(2741, 98)

In [9]:
# modeling_cols = ddict['train_x'].columns.tolist()[1:]
modeling_cols = ddict['train_x'].columns.tolist()

In [10]:
pred_y = model.predict(train_x[modeling_cols])

In [11]:
train_y = train_y.to_numpy()

In [12]:
evals, message = \
    regression_evals(y=train_y, p=pred_y, verbose=1)

In [13]:
message

'MAPE: 0.507407, R2SCORE: 0.561654'